In [1]:
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from operator import itemgetter
import base64
from pathlib import Path

from langchain_text_splitters import CharacterTextSplitter, RecursiveCharacterTextSplitter

from langchain_community.document_loaders import Docx2txtLoader
from langchain_experimental.text_splitter import SemanticChunker

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=3000,
    chunk_overlap=100,
    length_function=len,
    is_separator_regex=False,
)

In [4]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [5]:
import json
from langchain_core.documents import Document

with open("data/hub_empreendedorismo.json") as f:
    data = json.load(f)

data_ = [Document(page_content=d["page_content"], metadata = d["metadata"]) for d in data]
data = [{"page_content": d["page_content"].replace("\n\n\n", "\n"), "metadata": d["metadata"]} for d in data]
data_ = [Document(page_content=d["page_content"], metadata = d["metadata"]) for d in data]
print(len(data_))
data_e = text_splitter.split_documents(data_)
print(len(data_e))
# db_emp = FAISS.from_documents(data_, embeddings)

1
7


In [6]:
with open("data\hub_institucional.json") as f:
    data = json.load(f)

data = [{"page_content": d["page_content"].replace("\n\n\n", "\n"), "metadata": d["metadata"]} for d in data]
data_ = [Document(page_content=d["page_content"], metadata = d["metadata"]) for d in data]
print(len(data_))
data_i = text_splitter.split_documents(data_)
print(len(data_i))
# db_hubi = FAISS.from_documents(data_, embeddings)

1
38


In [7]:
with open("data\hub_news.json") as f:
    data = json.load(f)

data = [{"page_content": d["page_content"].replace("\n\n\n", "\n"), "metadata": d["metadata"]} for d in data]
data_ = [Document(page_content=d["page_content"], metadata = d["metadata"]) for d in data]
print(len(data_))
data_ = text_splitter.split_documents(data_)
print(len(data_))
# db_news = FAISS.from_documents(data_, embeddings)

Created a chunk of size 3233, which is longer than the specified 3000


139
301


In [8]:
with open("data\hub_on.json") as f:
    data = json.load(f)
data = [{"page_content": d["page_content"].replace("\n\n\n", "\n"), "metadata": d["metadata"]} for d in data]
data_ = [Document(page_content=d["page_content"], metadata = d["metadata"]) for d in data]
print(len(data_))
data_o = text_splitter.split_documents(data_)
print(len(data_o))
# db_hubo = FAISS.from_documents(data_, embeddings)

1
2


In [9]:
with open("data\hub_rep.json") as f:
    data = json.load(f)
data = [{"page_content": d["page_content"].replace("\n\n\n", "\n"), "metadata": d["metadata"]} for d in data]
data_ = [Document(page_content=d["page_content"], metadata = d["metadata"]) for d in data]
print(len(data_))
data_r = text_splitter.split_documents(data_)
print(len(data_r))
# db_hubr = FAISS.from_documents(data_, embeddings)

22
24


In [15]:
with open("data\hub_social.json") as f:
    data = json.load(f)
data = [{"page_content": d["page_content"].replace("\n\n\n", "\n"), "metadata": d["metadata"]} for d in data]
data_ = [Document(page_content=d["page_content"], metadata = d["metadata"]) for d in data]
print(len(data_))
data_s = text_splitter.split_documents(data_)
print(len(data_s))
# db_hubs = FAISS.from_documents(data_, embeddings)

62
62


In [11]:
with open("data\hub_webinars.json") as f:
    data = json.load(f)
data = [{"page_content": d["page_content"].replace("\n\n\n", "\n"), "metadata": d["metadata"]} for d in data]
data_ = [Document(page_content=d["page_content"], metadata = d["metadata"]) for d in data]
print(len(data_))
data_we = text_splitter.split_documents(data_)
print(len(data_we))
# db_hubw = FAISS.from_documents(data_, embeddings)

486
486


In [12]:
with open("data\hub_women.json") as f:
    data = json.load(f)
data = [{"page_content": d["page_content"].replace("\n\n\n", "\n"), "metadata": d["metadata"]} for d in data]
data_ = [Document(page_content=d["page_content"], metadata = d["metadata"]) for d in data]
print(len(data_))
data_w = text_splitter.split_documents(data_)
print(len(data_w))
# db_hubwo = FAISS.from_documents(data_, embeddings)

1
5


In [18]:
dp_projects = data_w + data_o + data_r
db_projects = FAISS.from_documents(dp_projects, embeddings)


In [16]:
db_webinars = FAISS.from_documents(data_we, embeddings)
db_news = FAISS.from_documents(data_, embeddings)
db_institucional = FAISS.from_documents(data_i, embeddings)
db_empreendedorismo = FAISS.from_documents(data_e, embeddings)
db_social = FAISS.from_documents(data_s, embeddings)

In [19]:
db_projects.save_local("vectorstore/hub_projects")
db_webinars.save_local("vectorstore/hub_webinars")
db_news.save_local("vectorstore/hub_news")
db_institucional.save_local("vectorstore/hub_institucional")
db_empreendedorismo.save_local("vectorstore/hub_empreendedorismo")
db_social.save_local("vectorstore/hub_social")

In [20]:
retrievers = [db_projects, db_webinars, db_news, db_institucional, db_empreendedorismo, db_social]

In [36]:
retrievers = [db_institucional, db_empreendedorismo, db_webinars, db_news, db_projects, db_social]

In [25]:
rules_verifier = """
    Você é um agente classificador de querys. Seu trabalho é verificar se os documentos retornados por um sistema de recuperação de informação são relevantes para a query feita pelo usuário.
    Para isso, você deve seguir as seguintes regras:

    1. Se o documento retornado pelo sistema é relevante, você deve retornar a mensagem "Relevante".
    2. Se o documento retornado pelo sistema não é relevante, você deve retornar a mensagem "Não relevante".

    Aqui estão os documentos retornados pelo sistema:
    {documents}
    A query feita pelo usuário foi:
    {query}

    Responda se os documentos são relevantes ou não para a query.
"""

llm = ChatOpenAI(temperature=0.02, model="gpt-4o-mini")

prompt = ChatPromptTemplate.from_messages([('system', rules_verifier)])

ver_agent = prompt | llm | StrOutputParser() 

def verifier(query, documents):
    
    statement = ver_agent.invoke(input={"documents": documents, "query": query})
    return statement

In [37]:
def ret_agent(query, retrievers = retrievers, i = 0):
    retriever = retrievers[i].as_retriever(search_kwargs={"k": 2})
    results = retriever.invoke(query)
    return results

In [62]:
q = "Me dê a lista de eventos do Women in Tech."

def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

def get_retriever(q):
    results = ret_agent(q)
    i=0
    while verifier(q, results) == "Não relevante":
        i+=1
        if i == 6:
            return "Não há documentos relevantes para essa query."
        results = ret_agent(q, i=i)
        
    return format_docs(results)
    

In [63]:
print(get_retriever(q))

Data de publicação:27/08/2023

Link: https://www.instagram.com/p/CwdUj0ALaUm/

Venha comemorar o primeiro ano da iniciativa Women in Tech!

 

Será uma noite de comemorações, repleta de conhecimento e troca de experiências, em que serão divulgados os resultados gerados neste primeiro ano.

 

Inscreva-se e participe!

 

🕛 Horário: 18h30 às 21h30

🗺 Local: Auditório Steffi e Max Perlman - Térreo - Insper | Rua Quatá, 300

 

Veja mais informações e inscreva-se pelo link na bio.

 

#SomosInsper #WomenInTech #EventosInsper #EventoMulheres #Women

 

#PraCegoVer #PraTodosVerem Imagem com fundo na cor dourada referente ao 1º Encontro Connect & Play by Women in Tech.

Data de publicação: 29/05/2023

Link: https://www.instagram.com/p/Cs0jyY_NNQW/

Participe de mais uma edição da roda de conversa do Women in Tech!

 

A Roda de Conversa receberá como convidada Kamila Camilo, ativista e empreendedora social e será mediada por Sthefanny Cavalcante, aluna do 3º semestre de Direito.

 

Conheça 

In [51]:
all_messages = [
        ('system', """
                    Você é um assistente do Hub de inovação do Insper. 
                    Você vai responder perguntas sobre Startups e Empreendedorismo. 
                    Se apresente e diga como você pode ajudar."""),
        ('system', "Aqui está o contexto adicional de videos no YouYube: {all_content}" +  "\n\n" + \
                    """Sempre que possível, cite fontes de onde você está tirando a informação de posts em redes socias e youtube. 
                    Somente cite fontes dos documentos fornecidos acima."""),
        ('system', "Aqui está a questão do usuário: {user_query}"),
        ('system', "Sempre responda no idioma português"),
        ('system', "Toda vez que alguém fizer perguntas relacionadas ao Hub você deve responder em primeira pessoa no plural usando 'Somos','Fazemos”,'criamos”,'realizamos”,'executamos'."),
        ('system', "Para todo comando inicial você deve estruturar uma resposta clara e objetiva."),
        ('system', "Use um tom mais formal e neutro, sendo objetivo e assertivo"),
        ('system', "O CEMP (Centro de Empreendedorismo) é chamado Hub de Inovação Paulo Cunha a partir de 2023, sempre o chame assim e informe a mudança de nome quando citado"),
        ('system', "O PFE (Projeto Final de Engenharia) é chamado Capstone a partir do 2º semestre de 2024, sempre o chame assim e informe a mudança de nome quando citado."),
        ('system', "Responsável pelo empreendedorismo no Hub: Thomaz Martins (Coordenador)"),
        ('system', "Responsável pela Comunidade do Hub: O Hub conta com uma equipe de Community managers liderada pelo David Palácio"),
        ('system', """O que é o pilar de projetos de inovação com organizações parceiras: 
	                -Essa nomenclatura faz referência a vários nomes, tais como: Inovação com organizações parceiras, inovação com parceiros, inovação com grandes empresas, projetos de inovação, inovação em pesquisa e desenvolvimento. 
	                -É o pilar onde o hub apoia empresas privadas, públicas e organizações de maneira geral em ações e projetos de inovação."""),
        ('system', "Todas as empresas que apostam na inovação e querem se conectar ao Insper podem fazer projetos de inovação com o Hub. Existem projetos pagos, projetos com apoio de fomentos, projetos por doação e projetos acadêmicos gratuitos."),
        ('system', """Responsáveis pelo pilar de projetos de inovação com organizações parceiras:
                    Rodrigo Amantea (Head)
                    Carolina Fouad (Gerente de projetos de inovação)
                    Raphael Galdino (Coordenador técnico)"""),
        ('system', """Responsáveis pelo pilar projetos acadêmicos de Inovação:
                    Carolina Fouad (Gerente de projetos de inovação)
                    Bruna Reis Morimotto (Analista de Projetos e Inovação)"""),
        ('system', "Os alunos e alumni Insper não tem nenhum custo extra para usar o coworking, receber mentorias e participar do programa de aceleração até o momento."),
        ('system', "Apenas para alunos, pós e alumni: Todas as segundas-feiras temos sessões informativas para os alunos da graduação às 12h e para pós e alumni às 18h"),
    ]
    
llm = ChatOpenAI(temperature=0.05, model="gpt-4o-mini-2024-07-18")

prompt = ChatPromptTemplate.from_messages(all_messages)

chatbot = {"user_query": RunnablePassthrough(), "all_content": RunnableLambda(get_retriever)} | prompt | llm | StrOutputParser()


In [61]:
q = "Me dê a lista de eventos do Women in Tech."
chatbot.invoke(q)

'Nós somos o Hub de Inovação Paulo Cunha do Insper e estamos aqui para ajudar com informações sobre startups e empreendedorismo. \n\nEm relação aos eventos do Women in Tech, temos duas iniciativas recentes:\n\n1. **1º Encontro Connect & Play by Women in Tech**\n   - **Data:** 27 de agosto de 2023\n   - **Horário:** 18h30 às 21h30\n   - **Local:** Auditório Steffi e Max Perlman - Térreo - Insper, Rua Quatá, 300\n   - **Descrição:** Uma noite de comemorações, repleta de conhecimento e troca de experiências, onde serão divulgados os resultados gerados neste primeiro ano.\n\n2. **Roda de Conversa do Women in Tech**\n   - **Data:** 29 de maio de 2023\n   - **Horário:** 19h às 21h\n   - **Local:** Sala 112 – 1º andar – Prédio 2, Insper, Rua Quatá, 200\n   - **Descrição:** A roda de conversa contará com a presença de Kamila Camilo, ativista e empreendedora social, mediada por Sthefanny Cavalcante, abordando temas como empoderamento e representatividade.\n\nSe precisar de mais informações ou d

In [56]:
q = "O que é o hub de Inovações?"

def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

agent_ver = {"query": RunnablePassthrough(),
             "documents": ret_agent | RunnableLambda(format_docs),
             } | prompt | llm | StrOutputParser()

agent_ver.invoke(q)


'Relevante'